# Installation


## Hugging face: https://huggingface.co/ai-forever/Real-ESRGAN

In [34]:
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

  Cloning https://github.com/sberbank-ai/Real-ESRGAN.git to /tmp/pip-req-build-15g8i83v
  Running command git clone --filter=blob:none --quiet https://github.com/sberbank-ai/Real-ESRGAN.git /tmp/pip-req-build-15g8i83v
  Resolved https://github.com/sberbank-ai/Real-ESRGAN.git to commit 362a0316878f41dbdfbb23657b450c3353de5acf
  Preparing metadata (setup.py) ... done


# Upscale images

In [38]:
from RealESRGAN import RealESRGAN
from PIL import Image
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "4"

model = RealESRGAN(device, scale=int(model_scale))
model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')

device: cpu


In [39]:
import os
from google.colab import files
import shutil
from io import BytesIO
import io
import tarfile

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
    buff = BytesIO()
    if '.png' in image_name.lower():
        img = img.convert('RGBA')
        img.save(buff, format='PNG')
    else:
        img.save(buff, format='JPEG')
    buff.seek(0)
    fp = io.BufferedReader(buff)
    img_tar_info = tarfile.TarInfo(name=image_name)
    img_tar_info.size = len(buff.getvalue())
    return img_tar_info, fp

def process_tar(path_to_tar):
    processing_tar = tarfile.open(path_to_tar, mode='r')
    result_tar_path = os.path.join('results/', os.path.basename(path_to_tar))
    save_tar = tarfile.open(result_tar_path, 'w')

    for c, member in enumerate(processing_tar):
        print(f'{c}, processing {member.name}')

        if not member.name.endswith(IMAGE_FORMATS):
            continue

        try:
            img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
            img_lr = Image.open(img_bytes, mode='r').convert('RGB')
        except Exception as err:
            print(f'Unable to open file {member.name}, skipping')
            continue

        img_sr = model.predict(np.array(img_lr))
        # adding to save_tar
        img_tar_info, fp = image_to_tar_format(img_sr, member.name)
        save_tar.addfile(img_tar_info, fp)

    processing_tar.close()
    save_tar.close()
    print(f'Finished! Archive saved to {result_tar_path}')

def process_input(filename):
    if tarfile.is_tarfile(filename):
        process_tar(filename)
    else:
        result_image_path = os.path.join('results/', os.path.basename(filename))
        image = Image.open(filename).convert('RGB')
        sr_image = model.predict(np.array(image))
        sr_image.save(result_image_path)
        print(f'Finished! Image saved to {result_image_path}')

#process_input("/content/low_resimg.png")

Finished! Image saved to results/low_resimg.png


## Provide video path to convert to HD

In [40]:
video_path = input("Provide input video_path ")

Provide input video_path/content/output_video_cropped.mp4


## Convert to frames
 The below code converts the video into frames and saves them in "ouptut_frames" dir

In [43]:
import cv2
import os

def convert_to_frames(video_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Couldn't open the video file")
        return

    ret, frame = cap.read()
    frame_count = 0

    while ret:
        frame_name = f"{frame_count}.jpg"
        frame_path = os.path.join(output_dir, frame_name)
        cv2.imwrite(frame_path, frame)

        ret, frame = cap.read()
        frame_count += 1

    cap.release()

    print(f"Frames saved to {output_dir}")

output_dir = "output_frames"
os.makedirs("output_frames", exist_ok=True)
convert_to_frames(video_path, output_dir)

Frames saved to output_frames


The below code sorts the filenames in frame order

In [44]:
def give_sorted_names(dirname):
  frames_list = os.listdir(dirname)
  if ".ipynb_checkpoints" in frames_list:
    frames_list.remove(".ipynb_checkpoints")
  #print(frames_list)
  sorted_list = sorted(frames_list, key=lambda x: int(x.split('.')[0]))
  return sorted_list

In [45]:
sorted_frames = give_sorted_names("output_frames")
print(sorted_frames)

['263.jpg', '240.jpg', '81.jpg', '1.jpg', '159.jpg', '174.jpg', '293.jpg', '5.jpg', '72.jpg', '258.jpg', '136.jpg', '94.jpg', '12.jpg', '243.jpg', '74.jpg', '252.jpg', '23.jpg', '244.jpg', '27.jpg', '182.jpg', '145.jpg', '116.jpg', '130.jpg', '247.jpg', '112.jpg', '76.jpg', '175.jpg', '192.jpg', '55.jpg', '292.jpg', '4.jpg', '133.jpg', '184.jpg', '108.jpg', '199.jpg', '79.jpg', '26.jpg', '49.jpg', '267.jpg', '157.jpg', '127.jpg', '31.jpg', '190.jpg', '95.jpg', '40.jpg', '132.jpg', '270.jpg', '152.jpg', '197.jpg', '38.jpg', '115.jpg', '135.jpg', '56.jpg', '210.jpg', '194.jpg', '167.jpg', '275.jpg', '0.jpg', '249.jpg', '111.jpg', '229.jpg', '110.jpg', '183.jpg', '142.jpg', '278.jpg', '266.jpg', '180.jpg', '212.jpg', '140.jpg', '211.jpg', '25.jpg', '268.jpg', '204.jpg', '271.jpg', '85.jpg', '179.jpg', '234.jpg', '117.jpg', '2.jpg', '93.jpg', '235.jpg', '52.jpg', '101.jpg', '131.jpg', '62.jpg', '32.jpg', '239.jpg', '14.jpg', '230.jpg', '201.jpg', '162.jpg', '198.jpg', '10.jpg', '147.jpg', 

## Looping through the list and upscaling every image

In [ ]:
for name in sorted_list:
  process_input(os.path.join("output_frames", name))

## Makes a list of HD frames to merge them

In [32]:
list_to_make_video = give_sorted_names("results")
print(list_to_make_video)

['1.jpg', '5.jpg', '4.jpg', '0.jpg', '2.jpg', '3.jpg']
['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg']


## Merges all the frames in order to make output_video.mp4

In [33]:
import cv2
import os

def images_to_video(image_folder, image_list, output_video):
    images = [os.path.join(image_folder, img) for img in image_list]
    image_path = images[0]
    frame = cv2.imread(image_path)
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video, fourcc, 1, (width,height))

    for image in images:
        video.write(cv2.imread(image))

    cv2.destroyAllWindows()
    video.release()


output_video = 'output_video.mp4'
images_to_video("results", list_to_make_video, output_video)
